In [14]:
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import util
from model_detection import Model
from model_detection import kalman_filter
from model_detection import pnp_detection
from model_detection import robust_matcher
from model_detection import Mesh
import time

In [5]:
def kalman_filter_params(n_states, n_measurements, n_inputs, dt):
    """
    returns predefines kalman filter parameters
    namely, measurement matrix and transition matrix

    params:
    n_states: Number of states
    n_measurements: number of measurements
    n_inputs: number of control actions
    dt: time between measurements

    """
    pass

In [68]:
print("Started....")

video_source = "./data/test/box.mp4"
model_path = "./data/test/cookies_ORB.yml"
mesh_path = "./data/test/box.ply"
# load the model
print("Parsing and registering model/mesh....")

model = Model.loadModel(model_path)
keypoints_model = model.getKeypoints()
descriptors_model = model.getDescriptors()
model_3d_points = model.get3DPoints()
# load mesh
mesh = Mesh.loadMesh(mesh_path)

print("Model/Mesh registration is done")

# intrinsic camera parameters
# fx, fy, cx, cy
camera_params = util.load_camera_parameters("data/calib.npy")

# init kalman filter
n_states = 18 # the number of states
n_measurements = 6 # the number of measured states
n_inputs = 0 # the number of control actions
dt = 0.125  #time between measurements (1/FPS)
kf = kalman_filter( params=kalman_filter_params( n_states, n_measurements, n_inputs, dt ) )

# init pnp_detection
pnp = pnp_detection(camera_params, method="iterative")
pnp_est = pnp_detection(camera_params)


# ransac parameters
ransac_confidence = 0.99
ransac_iterations = 500
max_reprojection_error = 6.0 # maximum allowed distance for inlier

# kalman parameters
kalman_min_inliers = 30

Started....
Parsing and registering model/mesh....
Model/Mesh registration is done


In [84]:
# initalize matcher
ratio_test = 0.7 # some default value
matcher = robust_matcher( ratio_test=ratio_test, feature_detector="ORB", matcher="FLANN", use_cross_check=False )

### main loop

In [85]:
# frame loop
frame_number = 0

cap = cv.VideoCapture(video_source)
# for online steaming
#cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera/no video presented.")
    exit()

while True:
    start_time = time.time()
    # Capture frame-by-frame
    ret, frame = cap.read()
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    # step 1 - match the points between the model and the frame
    matches, kp_frame = matcher.fastMatch(frame, keypoints_model, descriptors_model)
    
    print("matches number", len(matches))

    # step 2 - 3d-2d correspondencies
    points_2d_matches, points_3d_matches = [], []
    for i in range(len(matches)):
        points_2d_matches.append(kp_frame[ matches[i].queryIdx ].pt)
        points_3d_matches.append( model_3d_points[matches[i].trainIdx] )

    # draw outliers
    util.drawPoints( frame, points_2d_matches, color="red")
    
    cv.imshow('image window', frame)
    # add wait key. window waits until user presses a key
    cv.waitKey(0)
    # and finally destroy/close all open windows
    cv.destroyAllWindows()
    
    
    #time.sleep(1)

matches number 55
matches number 47
matches number 50
matches number 54
matches number 48
matches number 58


KeyboardInterrupt: 

In [45]:
kp_frame[matches[1].queryIdx].pt

(452.0, 111.0)

In [54]:
model_3d_points[0]

array([5.8165207, 6.5905914, 7.5      ], dtype=float32)